Link: https://www.youtube.com/watch?v=hztWQcoUbt0

In [66]:
import os
import json
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate # for qa model vs prompt template
import chromadb
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import numpy as np
from sentence_transformers import SentenceTransformer

import random
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import LambdaLR
from torch.nn.utils import clip_grad_norm
import matplotlib.pyplot as plt

In [67]:
MODEL = "llama3.1"

To optimize document retrieval effectively, a crucial component is having access to high-quality labeled data. This data typically consists of pairs matching user queries with their most relevant documents. While the ideal scenario would involve collecting and labeling this data from real worls user interactions and testing, for demonstration purposes we can simulate this data by generating potential queries for each chunk of our knowledge base.

In [68]:
loader = PyPDFLoader(file_path="./pdf2.pdf")
pages = loader.load()

doc = ""
for i in range(len(pages)):
    doc += pages[i].page_content

using a token based chunker. This will split the documents into manageable chunks to be embedded and retrieved. 

In [69]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=400
)
chunks = splitter.split_text(doc)
len(chunks)

81

In [70]:
chunks[1]

'tasks in the telecommunications field. We utilize a dataset with\n1,827 multiple-choice questions (MCQs) from 3GPP standard\ndocuments. A publicly available LLM named “Phi-2” is used to\nanswer the MCQs correctly. We develop a Retrieval-Augmented\nGeneration (RAG) pipeline to improve Phi-2 model’s perfor-\nmance. The RAG pipeline comprises document segmentation,\nsynthetic question-answer (QA) generation, custom fine-tuning\nof the embedding model, and incremental fine-tuning of Phi-\n2. Our experiments show that accuracy greatly increased by\ncombining all the above-mentioned steps in the RAG pipeline.\nThe proposed approach outperforms the baseline by 45.20%\nin terms of accuracy. This study identifies the limitations of\ninstruction fine-tuning in specialized fields and explores the'

As mentioned, it is best to use real testing data and labeling from your RAG application query/retrieval pairs - but for demonstration we will use some synthetic QA pair generation via an LLM.

The hope is that for each chunk of text that we have, we can create a possible user query that would most likely retrieve that chunk of text. This will allow us to further on down the line test the same user query, and assess based on retrieval of the expected chunk.

In [71]:
template = """
You are an AI assistant tasked with generating a single, realistic question-answer pair based on a given document. This question should be something a user might naturally ask when seeking information.

Given: {chunk}

Instructions:
1. Analyze the key topics, facts, and concepts in the given document, choose one to focus on.
2. Generate twenty similar questions that a user might ask to find the information in this document that does not contain any company name.
3. Use natural language and occasionally include typos or colloquialisms to mimic real user behavior in the question.
4. Ensure the question is semantically related to the document content without directly copying phrases.
5. Make sure that all of the questions are similar to each other i.e all asking about a similar topic/requesting the same information.

Output Format:
Reutrn a JSON object with the following structure:
```json
{{
    "question_1": "Generated question text",
    "question_2": "Generated question text",
    ...
}}
```

Be creative, think like a curious user, and generate your 20 similar questions that would naturally lead to the given document in a semantic search. Ensure your response is a valid JSON object content.
"""

prompt = ChatPromptTemplate.from_template(template)
llm = ChatOllama(temperature=1.0, model=MODEL)
chain = prompt | llm | JsonOutputParser()

In [72]:
chain.invoke({"chunk": chunks[1]})

{'question_1': 'How can we improve models for telecom-related tasks?',
 'question_2': "What's the best way to boost accuracy for telco-specific question-answering?",
 'question_3': 'Can you suggest any techniques for enhancing model performance in telecom domain?',
 'question_4': 'What are some strategies for improving LLMs in telecommunications tasks?',
 'question_5': 'How do we increase the effectiveness of models for telco-related MCQs?',
 'question_6': 'Are there any specific methods for fine-tuning models in telecom sector?',
 'question_7': "What's the most effective way to improve accuracy for telecom-specific questions?",
 'question_8': 'Can you provide some tips on how to enhance model performance for telco-related tasks?',
 'question_9': 'How can we leverage RAG pipeline for improving models in telecommunications field?',
 'question_10': 'What are the key factors that contribute to improved accuracy in telecom-specific question-answering?',
 'question_11': 'Can you explain how

In [73]:
parser = JsonOutputParser()

results = []

for i, chunk in tqdm(enumerate(chunks), total=len(chunks), desc="Processing Chunks"):
    output = chain.invoke({"chunk": chunk})
    results.append({
        "chunk_index": i,
        "input_chunk": chunk,
        "questions": output
    })

results


Processing Chunks: 100%|██████████| 81/81 [2:56:31<00:00, 130.76s/it]  


[{'chunk_index': 0,
  'input_chunk': 'Enhancing Large Language Models for Telecom\nNetworks Using Retrieval-Augmented Generation\nNasik Sami Khan, Md Mahibul Hasan, Md. Shamim Towhid, Saroj Basnet, Nashid Shahriar\nDepartment of Computer Science, University of Regina\n{nku618, mhr993, mty754, skb976, nashid.shahriar }@uregina.ca\nAbstract —This paper presents a comprehensive approach for\nfine-tuning large language models (LLMs) for domain-specific\ntasks in the telecommunications field. We utilize a dataset with\n1,827 multiple-choice questions (MCQs) from 3GPP standard\ndocuments. A publicly available LLM named “Phi-2” is used to\nanswer the MCQs correctly. We develop a Retrieval-Augmented\nGeneration (RAG) pipeline to improve Phi-2 model’s perfor-\nmance. The RAG pipeline comprises document segmentation,',
  'questions': {'question_1': 'How do I fine-tune large language models for telecom tasks?',
   'question_2': "What's the best approach for enhancing LLMs for domain-specific tele

In [74]:
output_file = "generated_questions.json"
with open(output_file, "w") as f:
    json.dump(results, f, indent=4)

In [75]:
with open("./generated_questions.json", 'r') as f:
    data = json.load(f)

reshaped_data = []
# having from ["question_1": "", "question_2": ""] to {question, chunk}
for entry in data:
    chunk = entry["input_chunk"]
    questions = entry["questions"]

    for key, question in questions.items():
        reshaped_data.append({
            "chunk": chunk,
            "question": question
        })


train_data, test_data = train_test_split(reshaped_data, test_size=0.2, shuffle=True, random_state=42)

# Save the training data to a JSON file
with open("train_data.json", "w") as train_file:
    json.dump(train_data, train_file, indent=4)

# Save the testing data to a JSON file
with open("test_data.json", "w") as test_file:
    json.dump(test_data, test_file, indent=4)

Let's setup the db:

In [76]:
client = chromadb.PersistentClient("./chromadb")
collection = client.get_or_create_collection(name="llm_collection", metadata={"hnsw:space": "cosine"})

embed all chunks to db:

In [77]:
i = 0
for chunk in chunks:
    collection.add(
        documents=[chunk],
        ids=[f"chunk_{i}"]
    )
    i+=1

Add of existing embedding ID: chunk_0
Insert of existing embedding ID: chunk_0
Add of existing embedding ID: chunk_1
Insert of existing embedding ID: chunk_1
Add of existing embedding ID: chunk_2
Insert of existing embedding ID: chunk_2
Add of existing embedding ID: chunk_3
Insert of existing embedding ID: chunk_3
Add of existing embedding ID: chunk_4
Insert of existing embedding ID: chunk_4
Add of existing embedding ID: chunk_5
Insert of existing embedding ID: chunk_5
Add of existing embedding ID: chunk_6
Insert of existing embedding ID: chunk_6
Add of existing embedding ID: chunk_7
Insert of existing embedding ID: chunk_7
Add of existing embedding ID: chunk_8
Insert of existing embedding ID: chunk_8
Add of existing embedding ID: chunk_9
Insert of existing embedding ID: chunk_9
Add of existing embedding ID: chunk_10
Insert of existing embedding ID: chunk_10
Add of existing embedding ID: chunk_11
Insert of existing embedding ID: chunk_11
Add of existing embedding ID: chunk_12
Insert of

In [78]:
# takes in embedding and retrieves the top 10 similar results from the database
def retrieve_documents_embeddings(query_embedding, k=10):
    query_embedding_list = query_embedding.tolist()

    results = collection.query(
        query_embeddings=[query_embedding_list],
        n_results=k
    )
    return results['documents'][0]

chromadb uses sentence-transformers/all-MiniLM-L6-v2 as base model. This technique is possible to use for any embedding model, so you'd want to assess your embedding model of choice here.

we use metrics:
* Mean Reciprocal Rank: measures how high the first correct answer appears in the list, on average. MRR is particularly useful when there's only one relevant item in the list or when we're primarily interested in the position of the first correct result.
* Recall@k: also known as hit rate measures the proportion of relevant items that are successfully retrieved within the top k results. In this context with one ground truth document, it's a binary measure that checks if the ground truth is present in the top k retrieved documents.

In [79]:
base_model = SentenceTransformer('all-MiniLM-L6-v2')

In [80]:
def reciprocal_rank(retrieved_docs, ground_truth, k):
    try:
        rank = retrieved_docs.index(ground_truth) + 1
        return 1.0 / rank if rank <= k else 0.0
    except ValueError:
        return 0.0

def hit_rate(retrieve_docs, ground_truth, k):
    return 1.0 if ground_truth in retrieve_docs[:k] else 0.0

In [81]:
def validate_embedding_model(test_data, base_model, k=10):
    hit_rates = []
    reciprocal_ranks = []

    for data_point in test_data:
        question = data_point["question"]
        ground_truth = data_point["chunk"]

        # generate embedding for the question
        question_embedding = base_model.encode(question)

        # retrieve documents using the embedding
        retrieved_docs = retrieve_documents_embeddings(question_embedding, k)

        # calculate metrics
        hr = hit_rate(retrieved_docs, ground_truth, k)
        rr = reciprocal_rank(retrieved_docs, ground_truth, k)

        hit_rates.append(hr)
        reciprocal_ranks.append(rr)

    # calculate average metrics
    avg_hit_rate = np.mean(hit_rates)
    avg_reciprocal_rank = np.mean(reciprocal_ranks)

    return {
        "average_hit_rate": avg_hit_rate,
        "average reciprocal rank": avg_reciprocal_rank
    }

print(validate_embedding_model(test_data, base_model))

{'average_hit_rate': 0.7037037037037037, 'average reciprocal rank': 0.368125367430923}


* with k=10, for 71.9% of the queries, the correct answer was found within the top 10 results.
* the reciprocal of 0.37 is about 2.7, indicating that on average, the first correct result appears at about 2.7 of 10.

This is how our baseline model performs. Now to train our embedding model so that it beats this score.

### Negative sampling
Negative sampling involves randomly selecting unrelated or irrelevant examples (called "negative samples") during the training process.

**Purpose**: It helps the model learn to distinguish between relevant and irrelevant information more effectively.

**Process**: Along with the correct (positive) matches for a query, the model is also shown randomly selected incorrect (negative) matches.

**Benefit**: This exposes the model to a wider range of examples, helping it develop a better understanding of what makes a good match versus a poor one.

**Efficiency**: It's a computationally cheap way to improve performance, as it doesn't require carefully curated negative examples.

By introducing these random negative samples, the model adapter better learns to create embeddings that not only bring relevant items closer together in the vector space, but also push irrelevant items further apart. This leads to more robust and discriminative embeddings, ultimately improving the model's ability to retrieve relevant information accurately.

These negative samples come into play during triplet loss.

### Triplet Loss
Triplet loss is a type of loss function used in various machine learning tasks, particularly in metric learning and embedding learning. Its primary goal is to learn embeddings such that similar examples are closer together in the embedding space while dissimilar examples are farther apart.

The triplet loss operates on triplets of data points:

Anchor (A): The reference sample
Positive (P): A sample similar to the anchor
Negative (N): A sample dissimilar to the anchor


This means the distance between the anchor and positive should be smaller than the distance between the anchor and negative, by at least the margin. The Negative document is dynamically randomly sampled from our NVIDIA form 10-K

In [82]:
# spits a random chunk from a pdf that is completely irrelevant to our main text
# just to show irrelevant data
unrelated_data_loader = PyPDFLoader("./unrelated.pdf")
unrelated_pages = unrelated_data_loader.load()

unrelated_doc = ""

for i in range(len(unrelated_pages)):
    unrelated_doc += unrelated_pages[i].page_content

unrelated_chunks = splitter.split_text(unrelated_doc)

def random_negative():
    random_sample = random.choice(unrelated_chunks)
    return random_sample

In [83]:
random_negative()

'VNF services. In this model, the elasticity overhead and the\ntrade-off between bandwidth and host resource consumption\nare considered together, while the previous works ignored this\nperspective of the problem. We propose a solution called Simple\nLazy Facility Location (SLFL) that optimizes the placement\nof VNF instances in response to on-demand workload. Our\nexperiments suggest that SLFL can accept two times more\nworkload while incurring similar operational cost compared to\nﬁrst-ﬁt and random placements.\nI. I NTRODUCTION\nNowadays, many Cloud Providers (CPs) such as Amazon\nAWS, Microsoft Azure and IBM Bluemix offer VNF as a\nservice (VNFaaS). An Enterprise Client (EC) can deploy all\nor part of its Network Functions (NFs) to cloud and enjoy'

Since our base model is making 384 dimension embeddings. So linear adapter is going to be 1 linear layer of 384x384 weight matrix and bias is going to be 384.

The goal here is to have a linear layer after the embedding model. So the embedding model does not get trained but the linear layer gets trained. This saves computational resources.

original embeddings that are produced by the base model are going to get pass through and then come through the linear adapter to predict appropriate chunk based on the question.

In [84]:
class LinearAdapter(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, input_dim)
    
    def forward(self, x):
        return self.linear(x)

The TripletDataset class is a custom dataset class that inherits from PyTorch's Dataset designed to work with triplet loss where each data point consists of three parts: a query, a positive example, and a negative example.

1. It retrieves the item at index idx from the training data.
2. Extracts the query and positive example from the item.
3. Uses the negative_sampler to generate a negative example.
4. Encodes the query, positive, and negative examples into embeddings using the base_model.
5. Returns the triplet of embeddings (query, positive, negative).

In [85]:
class TripletDataset(Dataset):
    def __init__(self, data, base_model, negative_sampler):
        self.data = data
        self.base_model = base_model
        self.negative_sampler = negative_sampler
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        item = self.data[index]
        query = item['question']
        positive = item['chunk']
        negative = self.negative_sampler()

        query_embed = self.base_model.encode(query, convert_to_tensor=True)
        positive_embed = self.base_model.encode(positive, convert_to_tensor=True)
        negative_embed = self.base_model.encode(negative, convert_to_tensor=True)

        return query_embed, positive_embed, negative_embed
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [86]:
def get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps):
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        
        # Ensure the learning rate decays linearly after warmup
        return max(0.0, float(num_training_steps - current_step) / float(max(1, num_training_steps - num_warmup_steps)))
    return LambdaLR(optimizer, lr_lambda)

def train_linear_adapter(base_model, train_data, negative_sampler, num_epochs=10, batch_size=32, learning_rate=2e-5, warmup_steps=100, max_grad_norm=1.0, margin=1.0):
    # initialize the LinearAdapter
    adapter = LinearAdapter(base_model.get_sentence_embedding_dimension()).to(device)

    # define loss function and optimizer
    triplet_loss = nn.TripletMarginLoss(margin=margin, p=2)
    optimizer = AdamW(adapter.parameters(), lr=learning_rate)

    # create dataset and dataloader
    dataset = TripletDataset(train_data, base_model, negative_sampler)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # calculate total number of training steps
    total_steps = len(dataloader) * num_epochs

    # create learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

    # training loop
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in dataloader:
            query_emb, positive_emb, negative_emb = [x.to(device) for x in batch]

            # forward pass
            adapted_query_emb = adapter(query_emb)

            # compute loss
            loss = triplet_loss(adapted_query_emb, positive_emb, negative_emb)

            # backward prop and optimization
            optimizer.zero_grad()
            loss.backward()

            # gradient clipping
            clip_grad_norm(adapter.parameters(), max_grad_norm)

            optimizer.step()
            scheduler.step()

            total_loss += loss.item()
        
        print(f"Epoch: {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader):.4f}")
    
    return adapter

In [87]:
# Define the kwargs dictionary
adapter_kwargs = {
    'num_epochs': 30,
    'batch_size': 32,
    'learning_rate': 0.003,
    'warmup_steps': 100,
    'max_grad_norm': 1.0,
    'margin': 1.0
}

# train the adapter using kwargs dictionary
trained_adapter = train_linear_adapter(base_model, train_data, random_negative, **adapter_kwargs)

save_dict = {
    'adapter_state_dict': trained_adapter.state_dict(),
    'adapter_kwargs': adapter_kwargs
}

C:\Users\nas596\AppData\Local\Temp\ipykernel_26216\1364301969.py:45: FutureWarning: `torch.nn.utils.clip_grad_norm` is now deprecated in favor of `torch.nn.utils.clip_grad_norm_`.
  clip_grad_norm(adapter.parameters(), max_grad_norm)


Epoch: 1/30, Loss: 0.6465
Epoch: 2/30, Loss: 0.1612
Epoch: 3/30, Loss: 0.0992
Epoch: 4/30, Loss: 0.0795
Epoch: 5/30, Loss: 0.0736
Epoch: 6/30, Loss: 0.0673
Epoch: 7/30, Loss: 0.0600
Epoch: 8/30, Loss: 0.0564
Epoch: 9/30, Loss: 0.0566
Epoch: 10/30, Loss: 0.0527
Epoch: 11/30, Loss: 0.0513
Epoch: 12/30, Loss: 0.0493
Epoch: 13/30, Loss: 0.0475
Epoch: 14/30, Loss: 0.0464
Epoch: 15/30, Loss: 0.0451
Epoch: 16/30, Loss: 0.0423
Epoch: 17/30, Loss: 0.0432
Epoch: 18/30, Loss: 0.0428
Epoch: 19/30, Loss: 0.0412
Epoch: 20/30, Loss: 0.0410
Epoch: 21/30, Loss: 0.0387
Epoch: 22/30, Loss: 0.0392
Epoch: 23/30, Loss: 0.0374
Epoch: 24/30, Loss: 0.0375
Epoch: 25/30, Loss: 0.0372
Epoch: 26/30, Loss: 0.0356
Epoch: 27/30, Loss: 0.0350
Epoch: 28/30, Loss: 0.0357
Epoch: 29/30, Loss: 0.0351
Epoch: 30/30, Loss: 0.0347


In [88]:
torch.save(save_dict, './params.pth')

loading the adapter

In [89]:
loaded_dict = torch.load('./params.pth')

# recreate the adapter
loaded_adapter = LinearAdapter(base_model.get_sentence_embedding_dimension())
loaded_adapter.load_state_dict(loaded_dict['adapter_state_dict'])

# access the training parameters
training_params = loaded_dict['adapter_kwargs']

for key, value in training_params.items():
    print(f"{key}: {value}")

num_epochs: 30
batch_size: 32
learning_rate: 0.003
warmup_steps: 100
max_grad_norm: 1.0
margin: 1.0


C:\Users\nas596\AppData\Local\Temp\ipykernel_26216\2619438742.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_dict = torch.load('./params.pth')


### Evaluate Adapter Performance. 

Below is the function to get the original embedding for the query and run it through our trained adapter.

In [90]:
def encode_query(query, base_model, adapter):
    device = next(adapter.parameters()).device
    query_emb = base_model.encode(query, convert_to_tensor=True).to(device)
    adapted_query_emb = adapter(query_emb)

    return adapted_query_emb.cpu().detach().numpy()

In [91]:
hit_rates = []
reciprocal_ranks = []

def evaluate_adapter(test_data, base_model, adapter, k=10):
    for data_point in test_data:
        question = data_point['question']
        ground_truth = data_point['chunk']

        # generate embedding for the question
        question_embedding = encode_query(question, base_model, adapter)
        # retrieve documents using the embedding
        retrieved_docs = retrieve_documents_embeddings(question_embedding, k)

        # calculate metrics
        hr = hit_rate(retrieved_docs, ground_truth, k)
        rr = reciprocal_rank(retrieved_docs, ground_truth, k)

        hit_rates.append(hr)
        reciprocal_ranks.append(rr)
    
    avg_hit_rate = np.mean(hit_rates)
    avg_reciprocal_rank = np.mean(reciprocal_ranks)

    return {
        'average hit rate': avg_hit_rate,
        'average reciprocal rank': avg_reciprocal_rank
    }

results = evaluate_adapter(test_data, base_model, loaded_adapter, k=10)
print(evaluate_adapter(test_data, base_model, loaded_adapter, k=10))

{'average hit rate': 0.7993827160493827, 'average reciprocal rank': 0.4916617675876934}
